In [5]:
import leancall as lc
import leancall.numpy

mod = lc.from_string("def foo x := x + 1")
type(mod.foo(3))


int

Does leancall change how I would go about pcode?


In [2]:
%%file /tmp/app.lean
import Lean.Data.Json
structure App where
    f : String
    args : List App
deriving Repr, Inhabited, BEq, Lean.ToJson, Lean.FromJson

def idapp (a : App) : Lean.Json := Lean.toJson a

partial def is_subterm (a b : App) : Bool :=
  if a == b then
    true
  else
    match a with
    | App.mk _ args => args.any (fun arg => is_subterm arg b)



Writing /tmp/app.lean


In [3]:
import leancall as lc
import leancall.numpy
from dataclasses import dataclass
@dataclass(frozen=True)
class App:
    f: str
    args: tuple["App", ...] = ()
    def of_dict(d):
        return App(d['f'], [App.of_dict(a) for a in d['args']])


@lc.register_from_lean("App")
def _(x) -> App:
    return x
mod = lc.from_file("/tmp/app.lean")
App.of_dict(mod.idapp(App("foo", [])))
x = App("x")
y = App("y")
f = lambda x: App("f", (x,))

#mod.is_subterm(f(x), x)  # true



calllean
look at other similar solutions?
prologs?
ocaml?
clr
https://pythonnet.github.io/pythonnet/python.html

https://github.com/codyroux/simple-prolog/blob/master/SimpleProlog/Basic.lean




 https://gymnasium.farama.org/introduction/basic_usage/
 https://github.com/codyroux/lean-controller

In [7]:
%%file /tmp/cartpole.lean

def action (obs : List Float) : Nat := 
    let pos := obs[0]!
    let vel := obs[1]!
    let ang := obs[2]!
    let ang_vel := obs[3]!
    -- I fiddled with this until it seems to work
    if pos - 1 * vel + 2.0 * ang + 0.7 * ang_vel < 0.0 then
        0
    else
        1


Overwriting /tmp/cartpole.lean


In [9]:
import numpy as np
mod = lc.from_file("/tmp/cartpole.lean")
mod.action([0.01234567, -0.00987654,  0.02345678,  0.01456789])

1

In [11]:
import gymnasium as gym

env = gym.make("CartPole-v1", render_mode="human")
observation, info = env.reset()

print(f"Starting observation: {observation}")
# Example output: [ 0.01234567 -0.00987654  0.02345678  0.01456789]
# [cart_position, cart_velocity, pole_angle, pole_angular_velocity]

print(type(observation))
episode_over = False
total_reward = 0
try:
    while not episode_over:
        # Choose an action: 0 = push cart left, 1 = push cart right
        action = mod.action(list(observation))
        observation, reward, terminated, truncated, info = env.step(action)
        # reward: +1 for each step the pole stays upright
        # terminated: True if pole falls too far (agent failed)
        # truncated: True if we hit the time limit (500 steps)

        total_reward += reward
        episode_over = terminated or truncated
finally:
    env.close()

print(f"Episode finished! Total reward: {total_reward}")


Starting observation: [-0.04921084 -0.01602873  0.02273309 -0.01083545]
<class 'numpy.ndarray'>
Episode finished! Total reward: 500.0


# Lean interact

In [68]:
def foo1(i):
    s = 0
    for _ in range(i):
        s += i
    return s


In [69]:
%%timeit
foo1(100000)

2.18 ms ± 5.57 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [63]:
%%file /tmp/foo.lean
def foo (i : Nat) : Nat := Id.run do
    let mut s := 0
    for _ in [0:i] do
        s := s + i
    return s

Overwriting /tmp/foo.lean


In [ ]:
mod = cl.from_file("/tmp/foo.lean")

Yea, it's constant time

In [75]:
mod.foo(100000000)

10000000000000000

In [74]:
%%timeit
mod.foo(100000000)

191 μs ± 65.4 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
from lean_interact import LeanREPLConfig, LeanServer, Command

config = LeanREPLConfig(verbose=True) # download and build Lean REPL
server = LeanServer(config) # start Lean REPL
server.run(Command(cmd="theorem ex (n : Nat) : n = 5 → n = 5 := id"))


In [14]:
server.run(Command(cmd="def mysucc x := x + 1", env=0))

CommandResponse(env=1)

In [15]:
server.run(Command(cmd="#eval mysucc 41", env=1))

CommandResponse(env=2, messages=[Message(start_pos=Pos(line=1, column=0), severity='info', data='42', end_pos=Pos(line=1, column=5))])

In [ ]:
"""
def to_lean(x : object) -> str:
    if isinstance(x, int):
        return str(x)
    elif isinstance(x, bool):
        return "true" if x else "false"
    elif isinstance(x, str):
        return f'"{x}"'
    elif isinstance(x, list):
        return "[" + ", ".join(map(to_lean, x)) + "]"
    elif isinstance(x, tuple):
        return "(" + ", ".join(map(to_lean, x)) + ")"
    elif isinstance(x, float):
        return str(x)
    # dict -> json or hashmap?
    else:
        raise Exception(f"Cannot convert {x} to Lean")
"""

Could have lean reach into python process and directly access memory for numpy / buffers
That'd be nasssssty.



In [ ]:
from lean_interact import LeanREPLConfig, LeanServer, Command
from functools import singledispatch

config = LeanREPLConfig(verbose=True) # download and build Lean REPL
server = LeanServer(config) # start Lean REPL

@singledispatch
def to_lean(x : object) -> str:
    raise Exception(f"Cannot convert {x} to Lean")
@to_lean.register
def _(x : int) -> str:
    return str(x)
@to_lean.register
def _(x : bool) -> str:
    return "true" if x else "false"
@to_lean.register
def _(x : str) -> str:
    return f'"{x}"'
@to_lean.register
def _(x : float) -> str:
    return str(x)
@to_lean.register(list)
def _(x : list) -> str:
    return "[" + ", ".join(map(to_lean, x)) + "]"
@to_lean.register(tuple)
def _(x : tuple) -> str:
    return "(" + ", ".join(map(to_lean, x)) + ")"

def from_lean(x : str, typ : str) -> object:
    if typ == "Nat" or typ == "Int":
        return int(x)
    elif typ == "Bool":
        assert x in ["true", "false"]
        return x == "true"
    elif typ == "String":
        return x.strip('"')
    elif typ == "Float":
        return float(x)
    elif typ == "Unit":
        assert x == "()"
        return ()
    elif typ.startswith('Option '):
        if x == "none":
            return None
        elif x.startswith("some "):
            inner_typ = typ[len('Option '):]
            return from_lean(x[len("some "):], inner_typ)
    else:
        # hail mary
        # may work for lists and tuples of simple elements
        return eval(x)
    
class LeanFun():
    def __init__(self, code, env=None):
        codesplit = code.split()
        assert codesplit[0] == "def"
        self.name = codesplit[1]
        self.code = code
        res = server.run(Command(cmd=code, env=env))
        if hasattr(res, "messages") and len(res.messages) > 0:
            for message in res.messages:
                if message.severity == "error":
                    raise Exception(message.data)
        self.env = res.env
        # get return type for deserialization
        res = server.run(Command(cmd=f"#check {self.name}", env=self.env))
        self.res_type = res.messages[0].data.split(":")[-1].strip()

    @classmethod
    def from_string(cls, code : str, env=None):
        ...

    def from_file(cls, filename : str, names : list[str], env=None):
        # load from file
        # return list of LeanFun objects
        ...

    def __call__(self, *args, **kwargs):
        res = server.run(Command(cmd=f"#eval {self.name} " + " ".join(map(to_lean, args)), env=self.env))
        assert len(res.messages) == 1
        message = res.messages[0]
        if message.severity == "info":
            return from_lean(message.data, self.res_type)
        else:
            raise Exception(message.data)
        
f = LeanFun("def mysucc (x y : Int) (s : String) (p : Nat × Nat) (r : List Int) (f : Float) := (f + 1.3, x + y + s.length + p.fst + r[0]!) |> some")
print(f.res_type)
f(1,2, "foo", (4,5), [1,2,3], 7.4)


Lake version 5.0.0-src+d8204c9 (Lean version 4.26.0)
Build completed successfully (26 jobs).
Option (Float × Int)


(8.7, 11)

In [74]:
%%prun -s cumulative
for i in range(1000):
    f(1,2, "foo", (4,5))

         216676 function calls (202767 primitive calls) in 0.260 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000    0.004    0.000    0.263    0.000 2346713141.py:90(__call__)
        1    0.001    0.001    0.259    0.259 <string>:1(<module>)
     1000    0.004    0.000    0.247    0.000 server.py:327(run)
     1000    0.003    0.000    0.211    0.000 server.py:242(run_dict)
     1000    0.003    0.000    0.189    0.000 server.py:173(_execute_cmd_in_repl)
     1000    0.002    0.000    0.173    0.000 threading.py:1115(join)
     1000    0.002    0.000    0.150    0.000 threading.py:1153(_wait_for_tstate_lock)
     1000    0.012    0.000    0.129    0.000 threading.py:1016(_bootstrap)
     1000    0.001    0.000    0.117    0.000 threading.py:1056(_bootstrap_inner)
     1000    0.003    0.000    0.114    0.000 ipkernel.py:744(run_closure)
     1000    0.001    0.000    0.086    0.000 threading.py:999(run)
     10

In [63]:
def mysucc(x, y):
    return x + y

In [64]:
%%timeit
mysucc(41, 43)

24 ns ± 0.404 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


https://github.com/leanprover-community/repl


Hmm. Ingest response via microlean?
Use simp

l.lean_simp



In [4]:
from kdrag.all import *
kd.lean("1 + 1.0")
import kdrag.printers.lean as plean

plean.of_expr

<function kdrag.printers.lean.of_expr(e: z3.z3.ExprRef)>